In [20]:
import numpy as np
import matplotlib.pyplot as plt
import deepxde as dde 
from analytical import sound_hard_circle


#dde.config.set_default_float('float64')

'''                        Problem parameters                               '''
k0 = 1            # wavenumber
wave_len = 2*np.pi / k0  # wavelength
radius = 1 * wave_len    # scatterer radius
d_air = 10 * wave_len     # distance between scatterer and absorbing layer

n_wave = 5  # number of mesh elements per wavelength

dim_x = 2 * (radius + d_air)

# The mesh element size is h_elem
h_elem = wave_len / n_wave

R = dim_x / 2.
nx = int(dim_x / h_elem)
print(nx)
print(dim_x / h_elem)


109
109.99999999999999


In [ ]:
obstacle = dde.geometry.Disk([0,0], radius)
disk = dde.geometry.Disk([0,0], R)

geom = dde.geometry.CSGDifference(disk, obstacle)

def pde(x, y):
    y0, y1 = y[:, 0:1], y[:, 1:2]
    
    y0_xx = dde.grad.hessian(y, x, component=0, i=0, j=0)
    y0_yy = dde.grad.hessian(y, x, component=0, i=1, j=1)

    y1_xx = dde.grad.hessian(y, x,component=1, i=0, j=0)
    y1_yy = dde.grad.hessian(y, x,component=1, i=1, j=1)

    return [-y0_xx - y0_yy - k0 ** 2 * y0,
            -y1_xx - y1_yy - k0 ** 2 * y1]


def boundary_obstacle(_, on_boundary):
    return on_boundary and obstacle.on_boundary(_)

def boundary_disk(_, on_boundary):
    return on_boundary and disk.on_boundary(_)

def func0(x):
    normal = disk.boundary_normal(x)
    w = 1j * k0 * np.exp(1j * k0 * x[:, 0:1])
    result = - w * normal[:, 0:1]
    return np.real(result)

def func1(x):
    normal = disk.boundary_normal(x)
    w = 1j * k0 * np.exp(1j * k0 * x[:, 0:1])
    result = - w * normal[:, 0:1]
    return np.imag(result)

bc_obstacle0 = dde.NeumannBC(geom, func0, boundary_obstacle, component = 0)
bc_obstacle1 = dde.NeumannBC(geom, func1, boundary_obstacle, component = 1)

bc_disk0 = dde.RobinBC(geom, lambda X, y: -k0 * y[:, 1:2], boundary_disk, component = 0)
bc_disk1 = dde.RobinBC(geom, lambda X, y: k0 * y[:, 0:1], boundary_disk, component = 1)



bcs = [bc_obstacle0, bc_obstacle1, bc_disk0, bc_disk1]
weights = [1, 1, 100, 100, 100, 100]
data = dde.data.PDE(geom, pde, bcs, num_domain= nx**2, num_boundary= 4 * nx, num_test= 10 * nx ** 2)
net = dde.maps.FNN([2] + [50] * 4 + [2], "tanh", "Glorot uniform")
model = dde.Model(data, net)

model.compile("adam", lr=0.001, loss_weights= weights)
model.train(epochs=10000)
model.compile("L-BFGS")
losshistory, train_state = model.train()
#dde.saveplot(losshistory, train_state, issave=True, isplot=True)


Compiling model...
Building feed-forward neural network...
'build' took 0.145320 s



/opt/conda/envs/DL/lib/python3.9/site-packages/tensorflow/python/keras/legacy_tf_layers/core.py:171: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` is deprecated and '
/opt/conda/envs/DL/lib/python3.9/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1692: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
2021-10-26 08:55:11.262795: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-10-26 08:55:11.464828: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: Quadro RTX 8000 computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 47.46GiB deviceMemoryBandwidth: 625.94GiB/s
2021-10-26 08:

'compile' took 2.773940 s

Initializing variables...


2021-10-26 08:55:14.003992: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 3000205000 Hz


Training model...

Step      Train loss                                                      Test loss                                                       Test metric
0         [2.38e-02, 6.10e-01, 7.10e-03, 1.81e-02, 5.66e+01, 2.49e+00]    [2.40e-02, 6.10e-01, 7.10e-03, 1.81e-02, 5.66e+01, 2.49e+00]    []  
1000      [5.50e-05, 6.06e-05, 3.71e-04, 1.52e-04, 1.16e-04, 1.32e-04]    [5.06e-05, 5.52e-05, 3.71e-04, 1.52e-04, 1.16e-04, 1.32e-04]    []  
2000      [2.70e-05, 3.62e-05, 8.31e-05, 2.56e-05, 5.00e-05, 5.13e-05]    [2.46e-05, 3.28e-05, 8.31e-05, 2.56e-05, 5.00e-05, 5.13e-05]    []  
3000      [1.50e-05, 2.27e-05, 2.24e-05, 6.70e-06, 2.62e-05, 2.41e-05]    [1.39e-05, 2.13e-05, 2.24e-05, 6.70e-06, 2.62e-05, 2.41e-05]    []  
4000      [9.52e-06, 1.47e-05, 6.64e-06, 3.58e-06, 1.34e-05, 1.29e-05]    [9.01e-06, 1.44e-05, 6.64e-06, 3.58e-06, 1.34e-05, 1.29e-05]    []  
5000      [6.00e-06, 1.08e-05, 3.56e-06, 3.87e-06, 9.99e-06, 1.11e-05]    [5.81e-06, 1.09e-05, 3.56e-06, 3.87e-06, 9